In [1]:
import os
import openai
import qdrant_client as qc
from qdrant_client.http.models import Batch
from qdrant_client.http import models
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

openai.api_key=os.environ["OPEN_AI_KEY"]

In [4]:
embedding_model = "text-embedding-ada-002"

ques_df = pd.read_csv("../data/interview_questions.csv")

questions = ques_df["Question"].tolist()

vector_size = 1536

def get_embedding(text: str) -> list:
    response = openai.Embedding.create(
        input=text,
        model=embedding_model,
    )
    return response["data"][0]["embedding"]

questions

['What experience do you have in leading a team of people?',
 'How would you handle a situation where two members of the crew are in disagreement over how to complete a task?',
 'What strategies do you use to ensure safety on board the vessel?',
 'How do you motivate and inspire your crew during long voyages or challenging conditions?',
 'What is your experience with navigation, charting, and plotting courses?',
 'Are you familiar with the latest technologies used for communication and navigation?',
 'How do you manage stress and fatigue while on duty?',
 'What steps do you take to prepare for unexpected weather changes or other emergencies?',
 'How do you handle conflicts between crew members?',
 'What strategies do you use to maintain morale among the crew?',
 'How do you stay informed about new developments in the industry?',
 'What experience do you have with managing budgets and resources?',
 'How do you ensure that all crew members are following safety protocols?',
 'What strateg

In [5]:
qc = qc.QdrantClient()

question_collection_name = "question_collection"


In [6]:
collections = qc.get_collections()
collections.collections[0].name

'question_collection'

In [7]:
if question_collection_name not in [collection.name for collection in collections.collections]:
    first_collection = qc.recreate_collection(
        collection_name=question_collection_name,
        vectors_config = models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
    )

In [10]:
for id, ques in enumerate(questions):
    # check
    query_filter = models.Filter(must=[models.HasIdCondition(has_id=[id])])
    res = qc.scroll(
        collection_name=question_collection_name,
        scroll_filter=query_filter,
    )
    if len(res[0]) > 0:
        continue
    print(res)
    qc.upsert(
        collection_name=question_collection_name,
        points=Batch(
            ids=[id],
            vectors=[get_embedding(ques)],
            payloads=[{"question": ques}],
        )
    )

([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)
([], None)

In [14]:
resume = "I am a conflict resolver"

# search for the most similar question

res = qc.search(
    collection_name=question_collection_name,
    query_vector=get_embedding(resume),
)

res

[ScoredPoint(id=86, version=86, score=0.8488132, payload={'question': 'How do you deal with conflict on your team?'}, vector=None),
 ScoredPoint(id=159, version=159, score=0.83758795, payload={'question': 'Have you ever had to deal with a workplace conflict, and if so, how did you resolve it?'}, vector=None),
 ScoredPoint(id=8, version=8, score=0.82465696, payload={'question': 'How do you handle conflicts between crew members?'}, vector=None),
 ScoredPoint(id=32, version=32, score=0.82185745, payload={'question': 'There is a conflict between two crew members. How do you handle it?'}, vector=None),
 ScoredPoint(id=82, version=82, score=0.7905544, payload={'question': 'Would you say you are a better verbal or written communicator?'}, vector=None),
 ScoredPoint(id=137, version=137, score=0.785191, payload={'question': 'Do you consider yourself a patient person?'}, vector=None),
 ScoredPoint(id=145, version=145, score=0.78091764, payload={'question': "ATC's must remain calm at all times. W